In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
from pyspark.sql.functions import approx_count_distinct,collect_list
from pyspark.sql.functions import collect_set,sum,avg,max,countDistinct,count
from pyspark.sql.functions import first, last, kurtosis, min, mean, skewness 
from pyspark.sql.functions import stddev, stddev_samp, stddev_pop, sumDistinct
from pyspark.sql.functions import variance,var_samp,  var_pop
from pyspark import SparkContext 
from pyspark.sql import SQLContext 
from pyspark.sql.types import StructType,StructField,StringType,IntegerType

import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml import PipelineModel
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.regression import LinearRegression
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import OneHotEncoder, StringIndexer, StringIndexerModel

# 初始化、导入processed data

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('taxi').getOrCreate()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

data = spark.read.csv('../input/taxi-data-processed/data_processed.csv', inferSchema=True, header=True)
zones = spark.read.csv('../input/taxi-data-processed/taxi_zones.csv', inferSchema=True, header=True)
# data_original = spark.read.csv('/kaggle/input/taxt-2019/taxi-data.csv', inferSchema=True, header=True)

In [ ]:
# data_original = data_original.withColumn("travel_time_seconds", col = data_original['tpep_dropoff_datetime'].cast('long')-data_original['tpep_pickup_datetime'].cast('long'))
# data_original = data_original.withColumn('pickup_week',col = F.dayofweek('tpep_pickup_datetime'))
# data_original = data_original.withColumn('dropoff_week',col = F.dayofweek('tpep_dropoff_datetime'))

In [ ]:
## store_and_fwd_flag    binary
def trans_store_and_fwd_flag(x):
    x_6 = 1 if x[6]=='Y' else 0
    temp = [x[0],x[1],x[2],x[3],x[4],x[5],x_6,x[7],x[8],x[9],x[10],x[11],x[12],x[13],x[14],x[15],x[16],x[17],x[18],x[19],x[20]]
    return temp


data = data.rdd.map(trans_store_and_fwd_flag).toDF(data.columns)
# data_original= data_original.rdd.map(trans_store_and_fwd_flag).toDF(data_original.columns)

# add day of year
data = data.withColumn('dayofyear',col = F.dayofyear('tpep_dropoff_datetime'))
# data_original = data_original.withColumn('dayofyear',col = F.dayofyear('tpep_dropoff_datetime'))


# add hour of day
data = data.withColumn('pickup_hour',col = F.hour(data.tpep_pickup_datetime))
data = data.withColumn('dropoff_hour',col = F.hour(data.tpep_dropoff_datetime))
# data_original = data_original.withColumn('pickup_hour',col = F.hour(data_original.tpep_pickup_datetime))
# data_original = data_original.withColumn('dropoff_hour',col = F.hour(data_original.tpep_dropoff_datetime))

In [ ]:
data

# EDA

## Histogram

* tip amount

In [ ]:
# rdd_tip_amount = data.select('tip_amount').rdd.flatMap(lambda x: x)
# bins, weights =rdd_tip_amount.histogram(10)
# plt.xlabel('tip amount')
# plt.ylabel('count (log)')
# plt.title('histogram of tip amount')
# plt.hist(bins[:-1],bins=bins,weights=np.log10(np.array(weights)+1),rwidth=0.8)

In [ ]:
# plt.figure(figsize = (10,10))
# plt.xlabel('tip amount')
# plt.ylabel('count (log)')
# plt.title('histogram of tip amount')
# plt.hist(bins[:-1],bins=bins,weights=np.log10(np.array(weights)+1),rwidth=0.8)

In [ ]:
# data.filter(data.tip_amount == 0).count()

In [ ]:
# data.count()

##  pearson corr

In [ ]:
# ori_feature_cols = ['VendorID',
# #  'tpep_pickup_datetime',
# #  'tpep_dropoff_datetime',
#  'passenger_count',
#  'trip_distance',
#  'RatecodeID',
#  'store_and_fwd_flag',
#  'PULocationID',
#  'DOLocationID',
#  'payment_type',
#  'travel_time_seconds',
#  'pickup_week',
#  'dropoff_week',
#  'fare_amount',
#  'tip_amount']

# ori_assembler = VectorAssembler(inputCols=ori_feature_cols,outputCol="features")
# ori_assembled_data = ori_assembler.transform(data)

# pearsonCorr = Correlation.corr(ori_assembled_data, 'features', 'pearson').collect()[0][0]
# pearsonCorr_np = pearsonCorr.toArray()

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.subplots(figsize = (18,18))
# sns.heatmap(pearsonCorr_np,xticklabels=ori_feature_cols ,yticklabels=ori_feature_cols, annot = True,vmax = 1,square = True,cmap = "Reds")
# plt.show()

fare_amount和tip_amount和trip_distance有明显的线性相关性；同时RatecodeID、travel_time和trip_distance也有明显的线性相关性。

注意到，fare_amount和tip_amount也有一定的线性相关性。

## 时间与费用的关系 分析

显然，trip_distance、travel_time和fare amount有正相关关系。因此我们研究 星期/时间段/特殊日子 对fare_amount/trip_distance, fare_amount/travel_time进行比较。

* 星期的影响

In [ ]:
# # pickup week -- fareamount
# pd_fee_per_miles_dayofweek = data.groupby('pickup_week').agg(F.mean(data.fare_amount / data.trip_distance)).toPandas()
# pd_fee_per_miles_dayofweek.rename(columns={'avg((fare_amount / trip_distance))':'avg_amount_per_miles'},inplace=True)
# pd_fee_per_miles_dayofweek.plot(x = 'pickup_week', y = 'avg_amount_per_miles',kind='scatter',figsize=(20,10))
# plt.title('average fare_amount per miles in each day of week')

In [ ]:
# # pickup week -- tip amount
# pd_tip_per_miles_dayofweek = data.groupby('pickup_week').agg(F.mean(data.tip_amount / data.trip_distance)).toPandas()
# pd_tip_per_miles_dayofweek.rename(columns={'avg((tip_amount / trip_distance))':'avg_amount_per_miles'},inplace=True)
# pd_tip_per_miles_dayofweek.plot(x = 'pickup_week', y = 'avg_amount_per_miles',kind='scatter',figsize=(20,10))
# plt.title('average tip_amount per miles in each day of week')

* 时间段的影响

In [ ]:
# # add hour of day
# data = data.withColumn('pickup_hour',col = F.hour(data.tpep_pickup_datetime))
# data = data.withColumn('dropoff_hour',col = F.hour(data.tpep_dropoff_datetime))

In [ ]:
# # pickup hour -- fare amount
# pd_fee_per_miles_pickhour = data.groupby('pickup_hour').agg(F.mean(data.fare_amount / data.trip_distance)).toPandas()
# pd_fee_per_miles_pickhour.rename(columns={'avg((fare_amount / trip_distance))':'avg_amount_per_miles'},inplace=True)
# pd_fee_per_miles_pickhour.plot(x = 'pickup_hour', y = 'avg_amount_per_miles',kind='scatter',figsize=(20,10))
# plt.title('average fare_amount per miles in each hour of day')

In [ ]:
# # droppff hour -- fare amount
# pd_fee_per_miles_drophour = data.groupby('dropoff_hour').agg(F.mean(data.fare_amount / data.trip_distance)).toPandas()
# pd_fee_per_miles_drophour.rename(columns={'avg((fare_amount / trip_distance))':'avg_amount_per_miles'},inplace=True)
# pd_fee_per_miles_drophour.plot(x = 'dropoff_hour', y = 'avg_amount_per_miles',kind='scatter',figsize=(20,10))
# plt.title('average fare_amount per miles in each hour (dropoff hour) of day')

In [ ]:
# # pickup hour -- tip amount
# pd_tip_per_miles_pickhour = data.groupby('pickup_hour').agg(F.mean(data.tip_amount / data.trip_distance)).toPandas()
# pd_tip_per_miles_pickhour.rename(columns={'avg((tip_amount / trip_distance))':'avg_amount_per_miles'},inplace=True)
# pd_tip_per_miles_pickhour.plot(x = 'pickup_hour', y = 'avg_amount_per_miles',kind='scatter')
# plt.title('average tip_amount per miles in each hour of day')

In [ ]:
# # droppff hour -- tip amount
# pd_tip_per_miles_drophour = data.groupby('dropoff_hour').agg(F.mean(data.tip_amount / data.trip_distance)).toPandas()
# pd_tip_per_miles_drophour.rename(columns={'avg((tip_amount / trip_distance))':'avg_amount_per_miles'},inplace=True)
# pd_tip_per_miles_drophour.plot(x = 'dropoff_hour', y = 'avg_amount_per_miles',kind='scatter')
# plt.title('average tip_amount per miles in each hour (dropoff hour) of day')

* 特殊日期的影响

In [ ]:
# # add day of year
# data = data.withColumn('dayofyear',col = F.dayofyear('tpep_dropoff_datetime'))

In [ ]:
# # pickup dayofyear -- fare_amount
# pd_fare_per_miles_dayofyear = data.groupby('dayofyear').agg(F.mean(data.fare_amount / data.trip_distance)).toPandas()
# pd_fare_per_miles_dayofyear.rename(columns={'avg((fare_amount / trip_distance))':'avg_amount_per_miles'},inplace=True)
# pd_fare_per_miles_dayofyear.plot(x = 'dayofyear', y = 'avg_amount_per_miles',kind='scatter')
# plt.title('average fare_amount per miles in each day of year')

In [ ]:
# pd_fare_per_miles_dayofyear.sort_values(by = 'avg_amount_per_miles', ascending = False).head(10)

实际上，大部分的数据都来自于19年12月的一小部分，data.filter(F.dayofyear(data.tpep_pickup_datetime) <= 300).count() == 188。是因为数据太少导致的误差。因此，日期对fare amount的影响不大。

### Zone 与 费用的影响

In [ ]:
# data_zones_temp = data.join(zones,data.PULocationID == zones.LocationID).selectExpr(
#     'DOLocationID',
#  'trip_distance',
# 'Borough as PUBorough',
#  'Zone as PUZone',
#  'service_zone as PUservice_zone',
#  'fare_amount',
#  'tip_amount',
# )

# data_zones = data_zones_temp.join(zones,data_zones_temp.DOLocationID==zones.LocationID).selectExpr(
#  'trip_distance',
#  'PUBorough',
#  'PUZone',
#  'PUservice_zone',
#  'Borough as DOBorough',
#  'Zone as DOZone',
#  'service_zone as DOservice_zone',   
#  'fare_amount',
#  'tip_amount',
# )

In [ ]:
# data_zones

* pick-up  Borough

In [ ]:
# groupby_PUB = data_zones.groupby('PUBorough')
# groupby_DOB = data_zones.groupby('DOBorough')

In [ ]:
# #pick up Borough  -- fare amount
# pd_PUBoro_fare_per_mile = groupby_PUB.agg(F.mean(data_zones.fare_amount / data_zones.trip_distance)).toPandas()
# pd_PUBoro_fare_per_mile.plot(x = 'PUBorough',kind='bar',figsize=(20,10))
# plt.title('fare_amount per mile of each pick-up Borough')

In [ ]:
# pd_PUBoro_fare_per_mile_Var = groupby_PUB.agg(F.variance(data_zones.fare_amount / data_zones.trip_distance)).toPandas()
# pd_PUBoro_fare_per_mile_Var.plot(x = 'PUBorough',kind='bar',figsize=(20,10))
# plt.title('variance of fare_amount per mile of each pick-up Borough')

In [ ]:
# pd_PUBoro_fare_per_mile

In [ ]:
# #pick up Borough -- tip amount
# pd_PUBoro_tip_per_mile = groupby_PUB.agg(F.mean(data_zones.tip_amount / data_zones.trip_distance)).toPandas()
# pd_PUBoro_tip_per_mile.plot(x = 'PUBorough',kind='bar',figsize=(20,10))
# plt.title('tip per mile of each pick-up Borough')

In [ ]:
# pd_PUBoro_tip_per_mile = groupby_PUB.agg(F.variance(data_zones.tip_amount / data_zones.trip_distance)).toPandas()
# pd_PUBoro_tip_per_mile.plot(x = 'PUBorough',kind='bar',figsize=(20,10))
# plt.title('variance of tip_amount per mile of each pick-up Borough')

* drop off Borough

In [ ]:
# #drop off Borough  -- fare amount
# pd_DOBoro_fare_per_mile = groupby_DOB.agg(F.mean(data_zones.fare_amount / data_zones.trip_distance)).toPandas()
# pd_DOBoro_fare_per_mile.plot(x = 'DOBorough',kind='bar')
# plt.title('fare_amount per mile of drop-off Borough')

* pick up Zone

In [ ]:
# groupby_PUZ = data_zones.groupby('PUZone')
# groupby_DOZ = data_zones.groupby('DOZone')

In [ ]:
# #pick up Zone -- fare amount
# pd_PUZone_fare_per_mile = groupby_PUZ.agg(F.mean(data_zones.fare_amount / data_zones.trip_distance)).toPandas()

In [ ]:
# pd_PUZone_fare_per_mile.sort_values('avg((fare_amount / trip_distance))',ascending=False).plot(x = 'PUZone',kind='bar',figsize=(20,10))
# plt.title('fare_amount/distance for each pick-up Zone')

我们惊奇的发现这些fare_amount/distance很高的三个区域，有一个共同点：没开多少路就要交$52.0

In [ ]:
# pd_PUZone_fare_per_mile.sort_values('avg((fare_amount / trip_distance))',ascending=False).head(30).plot(x='PUZone',kind='bar',figsize=(20,10))
# plt.title('TOP 30 expensive(fare_amount) pick-up Zone')

In [ ]:
# #pick up Zone -- tip amount

# pd_PUBoro_tip_per_mile = groupby_PUZ.agg(F.mean(data_zones.tip_amount / data_zones.trip_distance)).toPandas()

In [ ]:
# pd_PUBoro_tip_per_mile.sort_values('avg((tip_amount / trip_distance))',ascending=False).head(30).plot(x='PUZone',kind='bar',figsize=(20,10))
# plt.title('TOP 30 tip_wants pick-up Zone')

In [ ]:
# pd_PUBoro_tip_per_mile[(pd_PUBoro_tip_per_mile['PUZone']!= 'Pelham Bay Park') & (pd_PUBoro_tip_per_mile['PUZone']!='Arrochar/Fort Wadsworth' )]\
# .sort_values('avg((tip_amount / trip_distance))',ascending=False).plot(x='PUZone',kind='bar',figsize=(20,10))

# plt.title('tip_wants pick-up Zone expect outliers')

这里的outliers是因为有一些特殊的数据。

* drop off Zone 

* pick up Service Zone 

In [ ]:
# #pick up ServiceZone -- fare amount
# pd_PUServiceZone_fare_per_mile = data_zones.groupby('PUservice_zone').agg(F.mean(data_zones.fare_amount / data_zones.trip_distance)).toPandas()
# pd_PUServiceZone_fare_per_mile.plot(x = 'PUservice_zone',kind='bar')
# plt.title('fare_amount/distance for each pick-up Service Zone')

In [ ]:
# #pick up ServiceZone -- tip amount
# pd_PUServiceZone_tip_per_mile = data_zones.groupby('PUservice_zone').agg(F.mean(data_zones.tip_amount / data_zones.trip_distance)).toPandas()
# pd_PUServiceZone_tip_per_mile.plot(x = 'PUservice_zone',kind='bar')
# plt.title('tip_amount/distance for each pick-up Service Zone')

# Feature Engineering

In [ ]:
data_zones_temp = data.join(zones,data.PULocationID == zones.LocationID).drop('LocationID')

data_zones_temp = data_zones_temp.withColumnRenamed('Borough', 'PUBorough')
data_zones_temp = data_zones_temp.withColumnRenamed('Zone', 'PUZone')
data_zones_temp = data_zones_temp.withColumnRenamed('service_zone', 'PUservice_zone')

data_zones = data_zones_temp.join(zones,data_zones_temp.DOLocationID==zones.LocationID).drop('LocationID')

data_zones = data_zones.withColumnRenamed('Borough', 'DOBorough')
data_zones = data_zones.withColumnRenamed('Zone', 'DOZone')
data_zones = data_zones.withColumnRenamed('service_zone', 'DOservice_zone')

In [ ]:
# data_original_zones_temp = data_original.join(zones,data_original.PULocationID == zones.LocationID).drop('LocationID')

# data_original_zones_temp = data_original_zones_temp.withColumnRenamed('Borough', 'PUBorough')
# data_original_zones_temp = data_original_zones_temp.withColumnRenamed('Zone', 'PUZone')
# data_original_zones_temp = data_original_zones_temp.withColumnRenamed('service_zone', 'PUservice_zone')

# data_original_zones = data_original_zones_temp.join(zones,data_original_zones_temp.DOLocationID==zones.LocationID).drop('LocationID')

# data_original_zones = data_original_zones.withColumnRenamed('Borough', 'DOBorough')
# data_original_zones = data_original_zones.withColumnRenamed('Zone', 'DOZone')
# data_original_zones = data_original_zones.withColumnRenamed('service_zone', 'DOservice_zone')

## Strange Zones

In [ ]:
data_zones

In [ ]:
data_zones = data_zones.withColumn('isAstoriaPark',col= F.lit(0)) #8
data_zones = data_zones.withColumn('isRandallsIsland',col= F.lit(0)) #194
data_zones = data_zones.withColumn('isMaspeth',col= F.lit(0)) #157

In [ ]:
# data_original_zones = data_original_zones.withColumn('isAstoriaPark',col= F.lit(0)) #8
# data_original_zones = data_original_zones.withColumn('isRandallsIsland',col= F.lit(0)) #194
# data_original_zones = data_original_zones.withColumn('isMaspeth',col= F.lit(0)) #157

In [ ]:
def strange_zone_encoder(x):
    unchange = [x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8],x[9],x[10],x[11],x[12],x[13]
                ,x[14],x[15],x[16],x[17],x[18],x[19],x[20],x[21],x[22],x[23],x[24],x[25],x[26],x[27],x[28],x[29]]
    
    zones = [0,0,0]
    zone_pu = x[7]
    if zone_pu == 8:
      zones[0] = 1
    elif zone_pu == 194:
      zones[1] = 1
    elif zone_pu == 157:
      zones[2] = 1
    
    return unchange + zones

data_final = data_zones.rdd.map(strange_zone_encoder).toDF(data_zones.columns)
# data_original_final = data_original_zones.rdd.map(strange_zone_encoder).toDF(data_original_zones.columns)

## Encoder

In [ ]:
features_col = ['VendorID',
 'passenger_count',
 'trip_distance',
 'RatecodeID_',
 'store_and_fwd_flag',
 'payment_type_',
#  'fare_amount',
#  'total_amount',
 'travel_time_seconds', #
 'pickup_week', #
 'dropoff_week', #
 'dayofyear',
 'pickup_hour',
 'dropoff_hour',
 'PUBorough_',
 'PUservice_zone_',
 'DOBorough_',
 'DOservice_zone_',
 'isAstoriaPark',
 'isRandallsIsland',
 'isMaspeth']

labeling_col = ['PUBorough',
 'PUservice_zone',
 'DOBorough',
 'DOservice_zone']

labeling_col_out = ['PUBorough_',
 'PUservice_zone_',
 'DOBorough_',
 'DOservice_zone_']

one_hot_col = ['payment_type', 'RatecodeID']
one_hot_col_out = ['payment_type_', 'RatecodeID_']

### Zone - labeling encoder

In [ ]:
label_encoder = StringIndexer(inputCols=labeling_col, outputCols=labeling_col_out).setHandleInvalid("keep")

label_model=label_encoder.fit(data_final)
df = label_model.transform(data_final)

In [ ]:
# label_model=label_encoder.fit(data_original_final)
# df_original = label_model.transform(data_original_final)

### payment_type & RateCode_ID - One hot encoder

In [ ]:
onehot_encoder = OneHotEncoder(inputCols=one_hot_col, outputCols=one_hot_col_out)

label_model=onehot_encoder.fit(df)
df1 = label_model.transform(df)
#使用OneHotEncoder将分类变量转换为二进制稀疏向量

In [ ]:
# df_original = df_original.dropna()

In [ ]:
# label_model=onehot_encoder.fit(df_original)
# df_original1 = label_model.transform(df_original)

## Assemble

In [ ]:
assembler = VectorAssembler(inputCols=features_col,outputCol="features")
assembled_data = assembler.transform(df1)
data_ml = assembled_data.select('features','tip_amount','fare_amount')

In [ ]:
# assembled_data_original = assembler.transform(df_original1)
# data_ml_original = assembled_data_original.select('features','tip_amount','fare_amount')

In [ ]:
data_ml = data_ml.withColumn('tip_amount_is_zero',col = 1- (data_ml.tip_amount == 0).cast('int'))
# is zero, then 0;  none zero, then 1

# Model Training

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import GBTRegressor

In [ ]:
train, test = data_ml.randomSplit(weights = [0.70, 0.30], seed = 42)

## fare_amount Modeling

In [ ]:
# # gbt fit
# gbt_f = GBTRegressor(featuresCol="features", labelCol='fare_amount', maxDepth=5, maxIter=5, seed=42)
# gbt_f_Model = gbt_f.fit(train)

In [ ]:
# # gbt prediction
# predictions_fare = gbt_f_Model.transform(test)
# predictions_fare.select("prediction", "fare_amount", "features").show(20)

In [ ]:
# # see important features
# gbt_f_Model.featureImportances

In [ ]:
# gbt_model_show_string = gbt_f_Model.toDebugString

In [ ]:
# print(gbt_model_show_string)

In [ ]:
# features_col

In [ ]:
# # evaluate
# evaluator_mse = RegressionEvaluator(labelCol="fare_amount", predictionCol="prediction", metricName="mse")
# evaluator_mae = RegressionEvaluator(labelCol="fare_amount", predictionCol="prediction", metricName="mae")
# mse = evaluator_mse.evaluate(predictions_fare)
# mae = evaluator_mae.evaluate(predictions_fare)

# print("Mean Squared Error (MSE) on test data = %g" % mse)
# print("Mean Absolute Error (MAE) on test data = %g" % mae)


In [ ]:
# cnt = test.count()
# # predictions_fare_ = predictions_fare.withColumn('abs_error_variance', col = (F.abs(predictions_fare.prediction - predictions_fare.fare_amount) - mae)**2)
# ae_variance_fare = predictions_fare.withColumn('abs_error_variance', col = (F.abs(predictions_fare.prediction - predictions_fare.fare_amount) - mae)**2).groupby().sum('abs_error_variance').toPandas().iat[0, 0] / cnt

# print('Variance of abs error = %g' % ae_variance_fare)

In [ ]:
# ae_variance_fare

In [ ]:
# test.count()

In [ ]:
# variance of absolute errror : 
8111304.4856829485 /1819586

Overall, 用GBT预测fare amount on test data：
* MAE:   0.945968
* MSE:   5.35263
* variance of absolute error:  4.457774727703416

## tip_amount Modeling

先对tip是否是0进行一个逻辑回归二分类，之后再用GBT

In [ ]:
# logistic regression
lr_t = LogisticRegression(featuresCol="features", labelCol='tip_amount_is_zero')
lr_t_model = lr_t.fit(train)

In [ ]:
# logistic regression -- prediction
predictions_tip_classi = lr_t_model.transform(test).withColumnRenamed('prediction', 'prediction_is_tip_zero')
# predictions_tip_classi = predictions_tip_classi
predictions_tip_classi.select("prediction_is_tip_zero", "tip_amount", "features").show(40)

In [ ]:
# predictions_tip_classi.select("prediction_is_tip_zero", "tip_amount_is_zero", "features").show(40)

In [ ]:
# # evaluate of logistic regression
# from pyspark.ml.evaluation import BinaryClassificationEvaluator
# # BinaryClassificationEvaluatorMetricType = 'areaUnderROC'
# lr_evaluator = BinaryClassificationEvaluator(labelCol = 'tip_amount_is_zero', rawPredictionCol= 'prediction_is_tip_zero')
# auc = lr_evaluator.evaluate(predictions_tip_classi)
# print('AUC of logsitic regression on test data = %g' %auc)

In [ ]:
# # area under PR
# from pyspark.ml.evaluation import BinaryClassificationEvaluator
# lr_evaluator_1 = BinaryClassificationEvaluator(labelCol = 'tip_amount_is_zero', rawPredictionCol= 'prediction_is_tip_zero',metricName='areaUnderPR')
# auPR = lr_evaluator_1.evaluate(predictions_tip_classi)
# print('areaUnderPR of logsitic regression on test data = %g' %auPR)

AUC 好像不太靠谱，AUC和准确率没有直接相关性。

In [ ]:
# gbt fit
gbt_t = GBTRegressor(featuresCol="features", labelCol='tip_amount', maxDepth=5, maxIter=5, seed=42)
gbt_t_Model = gbt_t.fit(train)

In [ ]:
# gbt -- prediction
predictions_tip = gbt_t_Model.transform(test)
predictions_tip.select('prediction', "tip_amount", "features").show(20)

In [ ]:
# # evaluate of gbt
# evaluator_mse = RegressionEvaluator(labelCol="tip_amount", predictionCol="prediction", metricName="mse")
# evaluator_mae = RegressionEvaluator(labelCol="tip_amount", predictionCol="prediction", metricName="mae")
# mse = evaluator_mse.evaluate(predictions_tip)
# mae = evaluator_mae.evaluate(predictions_tip)

# print("Mean Squared Error (MSE) on test data = %g" % mse)
# print("Mean Absolute Error (MAE) on test data = %g" % mae)


In [ ]:
# # predictions_tip = predictions_tip.withColumn('abs_error_variance', col = (F.abs(predictions_tip.prediction_final - predictions_tip.tip_amount) - mae)**2)
# # ae_variance_tip = predictions_tip.groupby().sum('abs_error_variance')
# cnt = test.count()
# ae_variance_tip = predictions_tip.withColumn('abs_error_variance', col = (F.abs(predictions_tip.prediction - predictions_tip.tip_amount) - mae)**2).groupby().sum('abs_error_variance').toPandas().iat[0, 0] / cnt
# print('Variance of abs error = %g' % ae_variance_tip)

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id as mi
id=mi()
predictions_tip = predictions_tip.withColumn("match_id", id)
predictions_tip_classi_ = predictions_tip_classi.withColumn("match_id", id).select('match_id','prediction_is_tip_zero')


In [ ]:
prediction_tip_final = predictions_tip_classi_.join(predictions_tip,predictions_tip.match_id==predictions_tip_classi_.match_id, 'inner').drop(predictions_tip_classi_.match_id)
prediction_tip_final = prediction_tip_final.withColumn('prediction_final',col = prediction_tip_final.prediction_is_tip_zero * prediction_tip_final.prediction)
prediction_tip_final

In [ ]:
# evaluate of final model
evaluator_mse = RegressionEvaluator(labelCol="tip_amount", predictionCol="prediction_final", metricName="mse")
evaluator_mae = RegressionEvaluator(labelCol="tip_amount", predictionCol="prediction_final", metricName="mae")
mse = evaluator_mse.evaluate(prediction_tip_final)
mae = evaluator_mae.evaluate(prediction_tip_final)

print("Mean Squared Error (MSE) on test data = %g" % mse)
print("Mean Absolute Error (MAE) on test data = %g" % mae)

# predictions_tip_ = prediction_tip_.withColumn('abs_error_variance', col = (F.abs(prediction_tip_.prediction_final - prediction_tip_.tip_amount) - mae)**2)
# ae_variance_tip = predictions_tip_.groupby().sum('abs_error_variance')
cnt = test.count()
ae_variance_tip = prediction_tip_final.withColumn('abs_error_variance', col = (F.abs(prediction_tip_final.prediction_final - prediction_tip_final.tip_amount) - mae)**2).groupby().sum('abs_error_variance').toPandas().iat[0, 0] / cnt

print('Variance of abs error = %g' % ae_variance_tip)

# Linear Regression

In [ ]:
linear_assembler = VectorAssembler(inputCols=['trip_distance'],outputCol="features")
lr_assembled_data = linear_assembler.transform(data)
data_ml_for_lr = lr_assembled_data.select('features','tip_amount','fare_amount')

In [ ]:
data_ml_for_lr

In [ ]:
train_lr, test_lr = data_ml_for_lr.randomSplit(weights = [0.70, 0.30], seed = 42)

In [ ]:
lr = LinearRegression(regParam=0.0, labelCol = 'fare_amount')

lr_model = lr.fit(train_lr)
lr_model.coefficients

In [ ]:
prediction_lrreg = lr_model.transform(test_lr)

In [ ]:
prediction_lrreg

In [ ]:
# evaluate of gbt
from pyspark.ml.evaluation import RegressionEvaluator
evaluator_mse = RegressionEvaluator(labelCol="fare_amount", predictionCol="prediction", metricName="mse")
evaluator_mae = RegressionEvaluator(labelCol="fare_amount", predictionCol="prediction", metricName="mae")
mse = evaluator_mse.evaluate(prediction_lrreg)
mae = evaluator_mae.evaluate(prediction_lrreg)

print("Mean Squared Error (MSE) on test data = %g" % mse)
print("Mean Absolute Error (MAE) on test data = %g" % mae)

cnt = test_lr.count()
ae_variance = prediction_lrreg.withColumn('abs_error_variance', col = (F.abs(prediction_lrreg.prediction - prediction_lrreg.fare_amount) - mae)**2).groupby().sum('abs_error_variance').toPandas().iat[0, 0] / cnt

print('Variance of abs error = %g' % ae_variance)

test_results = lr_model.evaluate(test_lr)
print("R2: {}".format(test_results.r2))

# Interpretation

# Robustness -- apply model on the dirty data set

In [ ]:
# fare_predict_on_oridata= gbt_f_Model.transform(data_ml_original)
# tip_predict_on_oridata = gbt_t_Model.transform(data_ml_original)

In [ ]:
# from pyspark.ml.evaluation import RegressionEvaluator
# evaluator_mse = RegressionEvaluator(labelCol="fare_amount", predictionCol="prediction", metricName="mse")
# evaluator_mae = RegressionEvaluator(labelCol="fare_amount", predictionCol="prediction", metricName="mae")
# mse = evaluator_mse.evaluate(fare_predict_on_oridata)
# mae = evaluator_mae.evaluate(fare_predict_on_oridata)

# #fare amount
# print("Mean Squared Error (MSE) on test data = %g" % mse)
# print("Mean Absolute Error (MAE) on test data = %g" % mae)


In [ ]:
# evaluator_mse = RegressionEvaluator(labelCol="tip_amount", predictionCol="prediction", metricName="mse")
# evaluator_mae = RegressionEvaluator(labelCol="tip_amount", predictionCol="prediction", metricName="mae")
# mse = evaluator_mse.evaluate(tip_predict_on_oridata)
# mae = evaluator_mae.evaluate(tip_predict_on_oridata)

# # tip amount
# print("Mean Squared Error (MSE) on test data = %g" % mse)
# print("Mean Absolute Error (MAE) on test data = %g" % mae)